In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
def count_classes(data_dir):
    classes = os.listdir(data_dir)
    return len(classes), classes

In [ ]:
def model_color_detection_DSV3():
    batch_size = 32
    img_height = 224
    img_width = 224

    train_data_dir = '/content/drive/My Drive/dataset/WARNA_V2/training'
    val_data_dir = '/content/drive/My Drive/dataset/WARNA_V2/validation'

    num_classes_train, classes_train = count_classes(train_data_dir)
    num_classes_val, classes_val = count_classes(val_data_dir)

    print(f"Jumlah kelas dalam training data: {num_classes_train}")
    print(f"Kelas-kelas dalam training data: {classes_train}")

    print(f"\nJumlah kelas dalam validation data: {num_classes_val}")
    print(f"Kelas-kelas dalam validation data: {classes_val}")

    datagen_train = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )

    datagen_val = ImageDataGenerator(rescale=1./255)

    train_dataset = datagen_train.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    val_dataset = datagen_val.flow_from_directory(
        val_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
    )

    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes_train, activation='softmax')
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    #class MyCallback(tf.keras.callbacks.Callback):
    #    def on_epoch_end(self, epoch, logs={}):
    #       if logs.get('accuracy') > 0.90 and logs.get('val_accuracy') > 0.90:
    #            self.model.stop_training = True

    #my_callback = MyCallback()

    model.fit(train_dataset, epochs=50, validation_data=val_dataset)

    return model

In [ ]:
if __name__ == '__main__':
    #model = model_color_detection_DSV3()
    #model.save("/content/drive/My Drive/model_color_recognition_DSV3/model_color_recognition_DSV3.h5")

    # Load model
    model_path = "/content/drive/My Drive/model_color_recognition_DSV3/model_color_recognition_DSV3.h5"
    model = tf.keras.models.load_model(model_path)

    # Convert model to TensorFlow Lite
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    # Save the TensorFlow Lite model to a file
    tflite_model_path = "/content/drive/My Drive/MODEL_color_tflite/model_color_recognition_DSV3.tflite"
    with open(tflite_model_path, 'wb') as f:
        f.write(tflite_model)

    print(f"Model TensorFlow Lite berhasil disimpan di: {tflite_model_path}")

Model TensorFlow Lite berhasil disimpan di: /content/drive/My Drive/MODEL_color_tflite/model_color_recognition_DSV3.tflite
